In [1]:
import ee
from ee import batch
import pandas as pd
from StringIO import StringIO
import asset_paths_to_cop_data as Cop_filenames

In [2]:
import pkg_resources
pkg_resources.get_distribution("earthengine-api").version
# !pip install --upgrade pip
# !pip install 'earthengine-api==0.1.152' --force-reinstall
# !pip freeze

'0.1.152'

In [3]:
ee.Initialize()

In [4]:
sites = [
  "EMSR150",
  "EMSR277",
  "EMSR271",
  "EMSR273",
  "EMSR258",
  "EMSR122",
  "EMSR130"
  ]

In [6]:
def get_assets_as_FCs(filenames):
    return [(ee.FeatureCollection(roi), ee.FeatureCollection(flood), roi) for roi, flood in filenames]

In [9]:
def cutCookies(roi, fc):
    diff = ee.Feature(roi.first()).difference(fc.geometry(10), 10);
    return fc.merge(diff);

In [10]:
def export_as_asset(newFC, assetId):
    task = ee.batch.Export.table.toAsset(
        collection=newFC,
        assetId=assetId,
      )
    task.start()

In [11]:
def addFloodedClassLabels(feature):
    return feature.set({'isFlooded': 1})

def addNotFloodedClassLabels(feature):
    return feature.set({'isFlooded': 0})

In [12]:
def cut_site_and_export(site_FCs):
    for sub_site in site_FCs:
        roi = sub_site[0]
        flooded_area = sub_site[1]
        roi_id = sub_site[2]
        labelled_roi = roi.map(addNotFloodedClassLabels)
        labelled_flood = flooded_area.map(addFloodedClassLabels)
        newFC = cutCookies(labelled_roi, labelled_flood)
        asset_id = roi_id.replace('area_of_interest', 'newFC')
        export_as_asset(newFC, asset_id)

In [13]:
# # batch it up

# for site in sites_not_yet_uploaded:
#     filenames = Cop_filenames.ALL_FILENAMES[site]
#     site_FCs = get_assets_as_FCs(filenames)
#     cut_site_and_export(site_FCs)

In [14]:

roi = ee.FeatureCollection(pathwe_roi_id)
flooded_area = ee.FeatureCollection(pathwe_flood_id)
roi_id = pathwe_roi_id
labelled_roi = roi.map(addNotFloodedClassLabels)
labelled_flood = flooded_area.map(addFloodedClassLabels)
newFC = cutCookies(labelled_roi, labelled_flood)
asset_id = roi_id.replace('area_of_interest', 'newFC')
export_as_asset(newFC, asset_id)

In [15]:

roi = ee.FeatureCollection(irrawaddy_delta_roi_id)
flooded_area = ee.FeatureCollection(irrawaddy_delta_flood_id)
roi_id = irrawaddy_delta_roi_id
labelled_roi = roi.map(addNotFloodedClassLabels)
labelled_flood = flooded_area.map(addFloodedClassLabels)
newFC = cutCookies(labelled_roi, labelled_flood)
asset_id = roi_id.replace('area_of_interest', 'newFC')
export_as_asset(newFC, asset_id)